### Data Info
1) ATAC-seq peaks in human K562 cells.
2) Information on which Functional Genomic Region (FGR) the peak occurs (incl. coordinates of the FGR that you likely don't need).
3) Proteins localize to each peak, as identified by ChIP-seq (ENCODE databases). The value is ChIP-seq peak score. The higher the score, the more binding of the protein. "." means no binding (might be worth replacing "." with 0 or 1).


In [7]:
import pandas as pd
data = pd.read_csv('datasets/original_dataset.txt', sep='\t')
print(data.shape)

/tmp/ipykernel_249801/3824004981.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('datasets/original_dataset.txt', sep='\t')


(114004, 460)


In [8]:
#@title Data Cleaning
# Drop useless columns
data.drop(['peakName', 'K562_ATACseq', 'FGRstart', 'FGRend', 'FGR', 'FGRstrand', 'peakStart', 'peakEnd'], axis=1, inplace=True)

# Replace '.' with 0 for binding scores
data.replace('.', 0, inplace=True)

# Change columns starting with 'K562_' to dtype int
for col_name in data.columns:
  if 'K562_' in col_name:
    data[col_name] = data[col_name].astype(int)

# Remove outliers by keeping only peakScore between 10 to 2000
data = data[data['peakScore'] >= 10]
data = data[data['peakScore'] <= 2000]

# Remove rows having weird values for 'FGRstrand'
# data = data[data.FGRstrand.isin(['+', '-'])]

# Remove too sparsely valued columns
# non_zero_counts = (data!=0).sum()
# data = data.loc[:, non_zero_counts > 10000]
print(data.shape)

(113916, 452)


In [9]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
data.head(10).T

,0,1,2,3,4,5,6,7,8,9
chr,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1,chr1
peakScore,928,76,673,635,92,436,197,471,88,236
region,promoter,promoter,promoter,promoter,promoter,promoter,promoter,promoter,promoter,promoter
K562_AFF1_ENCSR241LIH,20,0,28,0,0,0,0,0,0,0
K562_AFF1_ENCSR426URK,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
K562_ZNF639_ENCSR949NVY,0,0,0,0,0,0,0,0,0,0
K562_ZNF830_ENCSR033NQK,0,0,0,0,0,0,0,0,0,0
K562_ZNF830_ENCSR851XLW,0,0,0,0,0,0,0,0,0,0
K562_ZSCAN29_ENCSR175SZH,52,0,59,0,0,86,577,0,0,0


In [10]:
print(data.groupby('region')['region'].count())

region
CPS               1398
TW                9145
divergent         2452
enhancer         10315
geneBody         48482
promoter         13512
untranscribed    28612
Name: region, dtype: int64


### Task 1

Regression using peakScore to predict all other proteins.


In [11]:
# Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import numpy as np

def linear_regression(X, y):
  scaler_X = StandardScaler()
  scaler_y = StandardScaler()
  X_scaled = scaler_X.fit_transform(X)
  y_scaled = scaler_y.fit_transform(y)
  X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

  model = LinearRegression()
  model.fit(X_train, y_train)
  y_pred_scaled = model.predict(X_test)
  y_pred = scaler_y.inverse_transform(y_pred_scaled)
  y_test_original = scaler_y.inverse_transform(y_test)

  mse = mean_squared_error(y_test_original, y_pred)
  r2 = r2_score(y_test_original, y_pred)

  return mse, r2

def scatter_plot(X, Y):
  X = np.array(X, dtype=np.float32)
  Y = np.array(Y, dtype=np.float32)
  plt.scatter(X, Y)
  plt.show()

df_cp = data.copy()

results = {}

for col_name in df_cp.columns[3:]:
  print(col_name)
  sub_df = df_cp[['region', 'peakScore', col_name]]
  sub_df = sub_df[(sub_df[col_name] >= 10) & (sub_df[col_name] <= 2000)]
  col_results = {}
  for region in sub_df['region'].unique():
    # print(region)
    sub_sub_df = sub_df[sub_df['region'] == region]
    num_samples = len(sub_sub_df)
    if num_samples <= 10:
      col_results[region] = {'R^2': None, 'MSE': None, 'num_samples': num_samples}
    else:
      mse, r2 = linear_regression(sub_sub_df[['peakScore']], sub_sub_df[[col_name]])
      col_results[region] = {'R^2': r2, 'MSE': mse, 'num_samples': num_samples}
  results[col_name] = col_results

  print("=================================================")


K562_AFF1_ENCSR241LIH
K562_AFF1_ENCSR426URK
K562_AGO1_ENCSR641BSL
K562_ARHGAP35_ENCSR571BUF
K562_ARID1B_ENCSR822CCM
K562_ARID2_ENCSR491EBY
K562_ARNT_ENCSR155KHM
K562_ARNT_ENCSR613NUC
K562_ARNT_ENCSR669NFS
K562_ASH1L_ENCSR115BBC
K562_ATF1_ENCSR000DNZ
K562_ATF2_ENCSR869IUD
K562_ATF3_ENCSR000BNU
K562_ATF3_ENCSR000DOG
K562_ATF3_ENCSR028UIU
K562_ATF4_ENCSR145TSJ
K562_ATF7_ENCSR972ZBV
K562_BACH1_ENCSR000EGD
K562_BCLAF1_ENCSR492LTS
K562_BCLAF1_ENCSR792IYC
K562_BCOR_ENCSR808AKZ
K562_BDP1_ENCSR000DOK
K562_BHLHE40_ENCSR000EGV
K562_BMI1_ENCSR782WRO
K562_BRCA1_ENCSR223MLH
K562_BRD4_ENCSR583ACG
K562_BRD9_ENCSR177XCS
K562_BRF1_ENCSR000DOJ
K562_BRF2_ENCSR000DOC
K562_C11orf30_ENCSR350XWY
K562_CBFA2T2_ENCSR699PVC
K562_CBFA2T3_ENCSR697YLJ
K562_CBX1_ENCSR948QLZ
K562_CBX2_ENCSR000ATU
K562_CBX3_ENCSR000ATV
K562_CBX3_ENCSR000BRT
K562_CBX5_ENCSR272JAT
K562_CBX8_ENCSR000ATW
K562_CC2D1A_ENCSR343IFJ
K562_CCAR2_ENCSR598GER
K562_CCNT2_ENCSR000DOA
K562_CDC5L_ENCSR121PFY
K562_CDK8
K562_CEBPB_ENCSR000BRQ
K562_CEBPB_

In [12]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Transform the R² and MSE values into DataFrames
r2_dict = {region: {col: metrics['R^2'] for col, metrics in cols.items()} for region, cols in results.items()}
mse_dict = {region: {col: metrics['MSE'] for col, metrics in cols.items()} for region, cols in results.items()}

# Create DataFrames from the dictionaries
r2_df = pd.DataFrame(r2_dict).transpose()  # R² DataFrame
mse_df = pd.DataFrame(mse_dict).transpose()  # MSE DataFrame


In [13]:
# Create dictionaries for R^2, MSE, and num_samples
r2_dict = {region: {col: metrics['R^2'] for col, metrics in cols.items()} for region, cols in results.items()}
mse_dict = {region: {col: metrics['MSE'] for col, metrics in cols.items()} for region, cols in results.items()}
samples_dict = {region: {col: metrics['num_samples'] for col, metrics in cols.items()} for region, cols in results.items()}

# Create DataFrames from the dictionaries
r2_df = pd.DataFrame(r2_dict).transpose()
mse_df = pd.DataFrame(mse_dict).transpose()
samples_df = pd.DataFrame(samples_dict).transpose()

# Stack the DataFrames and convert to long format
r2_long = r2_df.stack().reset_index()
mse_long = mse_df.stack().reset_index()
samples_long = samples_df.stack().reset_index()

# Rename columns for clarity
r2_long.columns = ['region', 'protein_name', 'R^2']
mse_long.columns = ['region', 'protein_name', 'MSE']
samples_long.columns = ['region', 'protein_name', 'num_samples']

# Merge all three long-format DataFrames into a single DataFrame
combined_df = r2_long.merge(mse_long, on=['region', 'protein_name'])
combined_df = combined_df.merge(samples_long, on=['region', 'protein_name'])

# Display the combined DataFrame
display(combined_df.head(10))  # Show the top 10 rows for inspection


,region,protein_name,R^2,MSE,num_samples
0,K562_AFF1_ENCSR241LIH,promoter,0.000875,67678.077578,2705.0
1,K562_AFF1_ENCSR241LIH,enhancer,0.002158,24802.590648,1320.0
2,K562_AFF1_ENCSR241LIH,divergent,-0.021476,43750.171498,350.0
3,K562_AFF1_ENCSR241LIH,geneBody,-0.000226,18791.120296,2593.0
4,K562_AFF1_ENCSR241LIH,CPS,-0.279766,114779.922939,81.0
5,K562_AFF1_ENCSR241LIH,TW,-0.006290,12241.380649,491.0
6,K562_AFF1_ENCSR241LIH,untranscribed,0.003117,36270.457202,1145.0
7,K562_AGO1_ENCSR641BSL,promoter,0.000096,38868.782297,5990.0
8,K562_AGO1_ENCSR641BSL,enhancer,-0.005480,36060.260364,1131.0
9,K562_AGO1_ENCSR641BSL,divergent,-0.011007,30701.081376,817.0


In [14]:
# Top 10 rows based on highest R^2 values
top_r2 = combined_df.sort_values(by='R^2', ascending=False).head(50)
print("Top 10 Rows by R^2:")
display(top_r2)

# # Top 10 rows based on lowest MSE values
# top_mse = combined_df.sort_values(by='MSE', ascending=True).head(10)
# print("\nTop 10 Rows by MSE:")
# print(top_mse)


Top 10 Rows by R^2:


,region,protein_name,R^2,MSE,num_samples
839,K562_HNRNPH1_ENCSR581CVA,enhancer,0.727870,1828.492759,19.0
2293,K562_ZBTB8A_ENCSR283ZNI,enhancer,0.670356,7200.087354,15.0
177,K562_CBX8_ENCSR000ATW,CPS,0.502080,56.541545,12.0
117,K562_BRCA1_ENCSR223MLH,geneBody,0.458347,5421.135458,153.0
1129,K562_MITF_ENCSR797SWM,geneBody,0.447658,514.920451,17.0
1516,K562_POLR2A_ENCSR031TFS,promoter,0.437488,995.106792,23.0
2295,K562_ZBTB8A_ENCSR283ZNI,untranscribed,0.349381,34019.463967,63.0
1748,K562_RNF2_ENCSR608XTF,CPS,0.346146,41396.477076,463.0
276,K562_CREBBP_ENCSR000ATT,untranscribed,0.341220,241046.968193,83.0
1629,K562_RBM14_ENCSR423FCW,CPS,0.314181,396.151275,33.0
